<a href="https://colab.research.google.com/github/Gauravsaxena816/LanguageDetection/blob/main/minorVOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np


In [2]:
import pandas as pd

file_path = "/content/minorbaka2.csv"
df = pd.read_csv(file_path)
df.head()

,mun_0,mup_0,phi_left,phi_right,Etrap,GB_tr,num_GBs,R2w,aplhaw,alphaR2w,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
0,-0.242156,-0.616386,1.734317,1.720126,0.325200,0.017967,0.020734,0.447364,0.097619,0.016528,...,0.372612,0.357603,0.309397,0.270523,0.233001,0.0,-0.067015,-0.314575,1.336861,-0.918013
1,-0.622296,-0.002881,0.326529,1.275277,0.817518,0.017967,0.020734,0.478055,-12.318245,-1.787862,...,0.150866,-0.411640,-0.921445,-1.346928,-1.744319,0.0,2.858036,0.047568,-0.732642,-2.305948
2,-0.212510,-0.605676,0.098329,-0.626600,-0.137485,0.017967,0.020734,0.478181,0.122384,0.016527,...,1.120037,1.160437,1.071156,1.012974,0.993404,0.0,-0.508762,3.870961,-0.359039,0.829447
3,2.747315,-0.606911,-0.445454,0.530829,-1.353394,0.017967,0.020734,0.473594,0.107626,0.016532,...,0.474957,0.196364,0.006030,-0.038752,-0.009162,0.0,-0.401878,-0.179169,0.712205,-0.717348
4,1.988060,2.496918,-0.306086,-0.171141,-0.524752,0.017967,0.020734,0.478202,0.091957,0.016509,...,0.386568,-0.116036,-0.600704,-0.957933,-1.229470,0.0,1.013316,-0.231165,-1.385160,1.443985


In [5]:
import numpy as np
# Handle missing values (if any)
df.fillna(df.median(), inplace=True) # Fill with median for numerical columns

# Identify and handle outliers (if needed)
# Example using IQR method for 'Jsc' and 'Voc'
Q1_jsc = df['Jsc'].quantile(0.25)
Q3_jsc = df['Jsc'].quantile(0.75)
IQR_jsc = Q3_jsc - Q1_jsc
lower_bound_jsc = Q1_jsc - 1.5 * IQR_jsc
upper_bound_jsc = Q3_jsc + 1.5 * IQR_jsc

Q1_voc = df['Voc'].quantile(0.25)
Q3_voc = df['Voc'].quantile(0.75)
IQR_voc = Q3_voc - Q1_voc
lower_bound_voc = Q1_voc - 1.5 * IQR_voc
upper_bound_voc = Q3_voc + 1.5 * IQR_voc

# Option 1: Remove outliers
df = df[~((df['Jsc'] < lower_bound_jsc) | (df['Jsc'] > upper_bound_jsc))]
df = df[~((df['Voc'] < lower_bound_voc) | (df['Voc'] > upper_bound_voc))]

# Option 2: Cap outliers
#df['Jsc'] = np.clip(df['Jsc'], lower_bound_jsc, upper_bound_jsc)
#df['Voc'] = np.clip(df['Voc'], lower_bound_voc, upper_bound_voc)


# Feature scaling (if needed)
# Example using StandardScaler for numerical features
numerical_cols = df.select_dtypes(include=np.number).columns
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df.head()


,mun_0,mup_0,phi_left,phi_right,Etrap,GB_tr,num_GBs,R2w,aplhaw,alphaR2w,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
0,-0.250384,-0.613208,1.728906,1.717492,0.316421,0.021479,0.024456,0.451630,0.093835,0.021630,...,0.378762,0.364796,0.316283,0.276560,0.237911,0.0,-0.063984,-0.313848,1.336064,-0.907609
1,-0.627869,0.003149,0.322278,1.272532,0.809073,0.021479,0.024456,0.482681,-10.784848,-1.733305,...,0.155947,-0.404846,-0.912902,-1.338901,-1.738598,0.0,2.869988,0.049222,-0.734985,-2.287964
2,-0.220945,-0.602449,0.094266,-0.629826,-0.146578,0.021479,0.024456,0.482809,0.115534,0.021629,...,1.129790,1.168047,1.077016,1.018098,0.998003,0.0,-0.507077,3.882405,-0.361103,0.830308
3,2.718204,-0.603689,-0.449068,0.527896,-1.363311,0.021479,0.024456,0.478168,0.102603,0.021633,...,0.481600,0.203474,0.013324,-0.032335,-0.004152,0.0,-0.399868,-0.178096,0.710941,-0.708040
4,1.964252,2.514573,-0.309815,-0.174252,-0.534107,0.021479,0.024456,0.482830,0.088874,0.021611,...,0.392786,-0.109088,-0.592593,-0.950385,-1.223960,0.0,1.019642,-0.230225,-1.387991,1.441489


In [19]:
# Split data into features (X) and target variables (y)
X = df.drop(['Voc'], axis=1)
y = df[['Voc']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#XGBoost Model
xgb_model = MultiOutputRegressor(XGBRegressor(random_state=42))
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# Evaluate XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

print("XGBoost Results:")
print(f"Mean Squared Error: {xgb_mse}")
print(f"R-squared: {xgb_r2}")




XGBoost Results:
Mean Squared Error: 0.00010684559674700722
R-squared: 0.9998894333839417


In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

#Decision Tree Model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

# Evaluate Decision Tree model
dt_mse = mean_squared_error(y_test, dt_pred)
dt_r2 = r2_score(y_test, dt_pred)

print("\nDecision Tree Results:")
print(f"Mean Squared Error: {dt_mse}")
print(f"R-squared: {dt_r2}")



Decision Tree Results:
Mean Squared Error: 1.0937663044146967e-06
R-squared: 0.9999988682838604


In [21]:
#Random Forest Model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Evaluate Random Forest Model
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print("\nRandom Forest Results:")
print(f"Mean Squared Error: {rf_mse}")
print(f"R-squared: {rf_r2}")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




Random Forest Results:
Mean Squared Error: 4.497468358070922e-07
R-squared: 0.9999995346485344


In [22]:
#SVM Model
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# Evaluate SVM Model
svm_mse = mean_squared_error(y_test, svm_pred)
svm_r2 = r2_score(y_test, svm_pred)

print("\nSVM Results:")
print(f"Mean Squared Error: {svm_mse}")
print(f"R-squared: {svm_r2}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().




SVM Results:
Mean Squared Error: 0.008499329873833394
R-squared: 0.9912057733401396


In [26]:
import plotly.graph_objects as go

# Define the model names and corresponding scores
models = ['XGBoost', 'Decision Tree', 'Random Forest', 'SVM']
scores = [xgb_r2, dt_r2, rf_r2, svm_r2]

fig = go.Figure(data=[go.Bar(
    x=models,
    y=scores,
    text=scores, #add the score values to the plot
    textposition='auto',
    marker_color=['crimson', 'lightblue', 'green', 'blue']
)])

fig.update_layout(
    title="Comparison of R-squared Scores for Different Regression Models (Voc)",
    xaxis_title="Models",
    yaxis_title="R-squared",
    yaxis_range=[0, 1.2],

)

fig.show()

In [27]:
mse_scores = {
    'XGBoost': xgb_mse,
    'Decision Tree': dt_mse,
    'Random Forest': rf_mse,
    'SVM': svm_mse
}
models = list(mse_scores.keys())
scores = list(mse_scores.values())

fig = go.Figure(data=[go.Bar(
    x=models,
    y=scores,
    text=scores,  # Add the score values to the plot
    textposition='auto',
    marker_color=['purple', 'red', 'orange', 'blue']  # Customize colors
)])

fig.update_layout(
    title="Comparison of MSE Scores for Different Regression Models (Voc)",
    xaxis_title="Models",
    yaxis_title="MSE",

)

fig.show()

In [28]:
import pandas as pd

# Assuming you have y_test and xgb_pred from your previous code

# Create a DataFrame for actual values
actual_df = pd.DataFrame({'Actual_Voc': y_test['Voc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Voc': xgb_pred.flatten()})

# Concatenate the two DataFrames
Voc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(Voc_df.head())

   Actual_Voc  Predicted_Voc
0   -0.820298      -0.826695
1   -1.597787      -1.594064
2   -1.552052      -1.548605
3   -1.452267      -1.454883
4   -0.096858      -0.096749


In [30]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Actual_Voc'],
    mode='markers',
    name='Actual',
    marker=dict(color='blue')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Predicted_Voc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Voc Values Xgboost",
    xaxis_title="Index",
    yaxis_title="Voc Values",
    legend_title="Data Type"
)

fig.show()

In [31]:
import pandas as pd

# Assuming you have y_test and xgb_pred from your previous code

# Create a DataFrame for actual values
actual_df = pd.DataFrame({'Actual_Voc': y_test['Voc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Voc': dt_pred.flatten()})

# Concatenate the two DataFrames
Voc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(Voc_df.head())

   Actual_Voc  Predicted_Voc
0   -0.820298      -0.820298
1   -1.597787      -1.597787
2   -1.552052      -1.552052
3   -1.452267      -1.452267
4   -0.096858      -0.096858


In [33]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Actual_Voc'],
    mode='markers',
    name='Actual',
    marker=dict(color='blue')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Predicted_Voc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='green')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Voc Values DT",
    xaxis_title="Index",
    yaxis_title="Voc Values",
    legend_title="Data Type"
)

fig.show()

In [34]:
import pandas as pd

# Assuming you have y_test and xgb_pred from your previous code

# Create a DataFrame for actual values
actual_df = pd.DataFrame({'Actual_Voc': y_test['Voc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Voc': rf_pred.flatten()})

# Concatenate the two DataFrames
Voc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(Voc_df.head())

   Actual_Voc  Predicted_Voc
0   -0.820298      -0.820298
1   -1.597787      -1.597870
2   -1.552052      -1.552260
3   -1.452267      -1.452267
4   -0.096858      -0.096858


In [36]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Actual_Voc'],
    mode='markers',
    name='Actual',
    marker=dict(color='red')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Predicted_Voc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='green')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Voc Values RF",
    xaxis_title="Index",
    yaxis_title="Voc Values",
    legend_title="Data Type"
)

fig.show()

In [38]:
import pandas as pd

# Assuming you have y_test and xgb_pred from your previous code

# Create a DataFrame for actual values
actual_df = pd.DataFrame({'Actual_Voc': y_test['Voc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Voc': svm_pred.flatten()})

# Concatenate the two DataFrames
Voc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(Voc_df.head())

   Actual_Voc  Predicted_Voc
0   -0.820298      -0.807146
1   -1.597787      -1.683200
2   -1.552052      -1.547892
3   -1.452267      -1.480930
4   -0.096858      -0.055840


In [40]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Actual_Voc'],
    mode='markers',
    name='Actual',
    marker=dict(color='blue')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=Voc_df.index[:1000],  # Use DataFrame index for x-axis
    y=Voc_df['Predicted_Voc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='orange')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Voc Values SVM",
    xaxis_title="Index",
    yaxis_title="Voc Values",
    legend_title="Data Type"
)

fig.show()